# 1: Introduction To Validation(Demo)

In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

admissions = pd.read_csv("admissions.csv")
admissions["actual_label"] = admissions["admit"]
admissions = admissions.drop("admit", axis=1)
print(admissions.head())

        gpa         gre  actual_label
0  3.177277  594.102992             0
1  3.412655  631.528607             0
2  2.728097  553.714399             0
3  3.093559  551.089985             0
4  3.141923  537.184894             0


# 2. Holdout Validation
Instructions Use the NumPy random.permutation function to randomize the index for the admissions Dataframe.
Use the loc[] method on the admissions Dataframe to return a new Dataframe in the randomized order. Assign this Dataframe to shuffled_admissions.
Select rows 0 to 514 (including row 514) from shuffled_admissions and assign to train.
Select the remaining rows and assign to test.
Finally, display the first 5 rows in shuffled_admissions.
Hint
When calling the iloc[] method, recall that the value at the ending index value isn't returned. iloc[0:515] will return rows 0 to 514.

In [3]:
import numpy as np
np.random.seed(8)
shuffled_index = np.random.permutation(admissions.index)
shuffled_admissions = admissions.loc[shuffled_index]

train = shuffled_admissions.iloc[0:515]

test = shuffled_admissions.iloc[515:len(shuffled_index)]

print(shuffled_admissions)

          gpa         gre  actual_label
260  3.414036  577.665610             0
173  2.932147  564.798764             0
256  2.674040  599.895858             0
167  2.923581  622.524665             0
400  3.414376  704.934217             1
415  3.512116  653.744260             1
111  3.011533  578.212886             0
428  3.552490  667.320643             1
312  3.243188  665.619332             0
516  4.000000  544.159398             1
481  2.975316  601.392062             1
264  3.150268  444.223279             0
424  3.424274  677.555275             1
208  3.733697  524.504412             0
308  3.171177  497.847973             0
213  3.202666  576.551696             0
603  3.077075  713.399147             1
467  3.348166  604.252785             1
612  3.511204  618.929875             1
557  3.725919  641.372775             1
502  3.807618  617.107327             1
92   2.771824  549.111663             0
66   3.027542  595.457519             0
219  3.252544  678.615733             0


# 3. Accuracy
Instructions Train a logistic regression model using the gpa column from the train Dataframe.
Use the LogisticRegression method predict to return the predicted labels for the gpa column from the test Dataframe. Assign the resulting list of labels to the predicted_label column in the test Dataframe.
Calculate the accuracy of the predictions by dividing the number of rows where actual_label matches predicted_label by the total number of rows in the test set.
Assign the accuracy value to accuracy and display it using the print function.
Hint
test["predicted_label"] == test["actual_label"] returns the index values for the rows where predicted_label matches actual_label.

In [4]:
logistic_model = LogisticRegression()
logistic_model.fit(train[["gpa"]], train["actual_label"])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [5]:
predicted_label = logistic_model.predict(test[["gpa"]])

In [6]:
test["predicted_label"] = predicted_label
test.head()

//anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,gpa,gre,actual_label,predicted_label
117,3.219669,483.761856,0,0
121,2.799225,654.216375,0,0
214,3.247795,518.556697,0,0
452,3.567595,591.399738,1,0
447,3.448206,546.646023,1,0


In [7]:
test["predicted_label"].value_counts()

0    122
1      7
Name: predicted_label, dtype: int64

In [8]:
match = test["predicted_label"] == test["actual_label"]
correct_predictions = match[test["predicted_label"] == test["actual_label"]]
correct_predictions.head()

117    True
121    True
214    True
324    True
252    True
dtype: bool

In [9]:
correct_predictions.count()

82

In [10]:
test.count()

gpa                129
gre                129
actual_label       129
predicted_label    129
dtype: int64

In [11]:
accuracy = 82 / 129
print(accuracy)

0.6356589147286822


# 4. Sensitivity And Specificity
Instructions Calculate the sensitivity value for the predictions on the test set and assign to sensitivity.
Calculate the specificity value for the predictions on the test set and assign to specificity.
Display both values using the print function.
Hint
Sensitivity = (# of True Positives) / (# of True Positives + # of False Negatives)
Specificity = (# of True Negatives) / (# of False Positives + # of True Negatives)
True Positive : True refers to predicted_label matching actual_label while Positive refers to actual_label being True (or 1).

In [12]:
model = LogisticRegression()
model.fit(train[["gpa"]], train["actual_label"])
labels = model.predict(test[["gpa"]])
test["predicted_label"] = labels
matches = test["predicted_label"] == test["actual_label"]
correct_predictions = test[matches]
accuracy = len(correct_predictions) / len(test)
print(accuracy)

0.6356589147286822


//anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
true_positive_filter = (test["predicted_label"] == 1) & (test["actual_label"] == 1)
true_positives = len(test[true_positive_filter])
false_negative_filter = (test["predicted_label"] == 0) & (test["actual_label"] == 1)
false_negatives = len(test[false_negative_filter])

sensitivity = true_positives / (true_positives + false_negatives)
print(sensitivity)

0.08333333333333333


In [14]:
false_positive_filter = (test["predicted_label"] == 1) & (test["actual_label"] == 0)
false_positives = len(test[false_positive_filter])
true_negative_filter = (test["predicted_label"] == 0) & (test["actual_label"] == 0)
true_negatives = len(test[true_negative_filter])

specificity = (true_negatives) / (false_positives + true_negatives)
print(specificity)

0.9629629629629629


# 5. ROC Curve
Instructions Import the relevant scikit-learn package you need to calculate the ROC curve.
Use the model to return predicted probabilities for the test set.
Use the roc_curve function to return the FPR and TPR values for different thresholds.
Create and display a line plot with:
the FPR values on the x-axis and
the TPR values on the y-axis.
Hint
Documentation on predict_proba. Documentation on roc_curve.

In [26]:
import matplotlib.pyplot as plt
from sklearn import metrics

probabilities = model.predict_proba(test[["gpa"]])
fpr, tpr, thresholds = metrics.roc_curve(test["actual_label"], probabilities[:,1])
plt.plot(fpr, tpr)
plt.show()

# 6. Area Under The Curve
Instructions Calculate the AUC score for our model on the training set and assign to auc_score.
Use the print function to display auc_score.
Hint
Documentation on roc_auc_score

In [30]:
from sklearn.metrics import roc_auc_score
auc_score = metrics.roc_auc_score(test["actual_label"], probabilities[:,1])
print(auc_score)

0.577932098765
